# Generate Lists for Neural Net

In [85]:
import csv
import bq
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sb

In [86]:
client = bq.Client.Get()

def Query(q):
    t0 = time.time()
    answer = client.ReadTableRows(client.Query(q)['configuration']['query']['destinationTable'])
    print 'Query time: ' + str(time.time() - t0) + ' seconds.'
    return answer

## Get Vessels that Have Been Matched to the CLAV List

In [87]:
q = '''select shiptype, count(*) num from [world-fishing-827:CLAV_match_results.v7_results] group by shiptype
       order by num desc '''
geartypes = Query(q)

Waiting on bqjob_r20abb86f3feb17de_0000015854b4c333_14 ... (0s) Current status: DONE   
Query time: 2.08411192894 seconds.


In [88]:
for g in geartypes:
    print g[0],g[1]

Longliners 2057
Tuna longliners 975
Purse seiners 412
Fish carriers 406
Trawlers 392
Tuna purse seiners 282
Pole and Line vessels 203
Non-fishing vessels nei 146
Unknown 127
Trollers 86
Recreational fishing vessels 52
Multipurpose vessels 47
Support Vessels 42
Bunkers 41
Fishery research vessels 37
Gillnetters 30
Multipurpose vessels nei 28
Dredgers 8
None 5
Handliners 4
Liners nei 3
Liners 3
Motherships 2
Seiners 2
Stern trawlers 1
Seiners nei 1
Other line vessels 1


## Rules for CLAV Shiptypes

In [89]:
clav_rules = {
'Longliners': 'Drifting longlines',
'Tuna longliners': 'Drifting longlines',
'Purse seiners': 'Purse seiners',
'Trawlers': 'Trawlers',
'Trawler':'Trawlers',
'Trollers':'Trollers' ,
'Pole and Line vessels': 'Pole and line',
'Tuna purse seiners':'Purse seiners',
'Gillnetters':'Set gillnets'}
    
# ignore the following cageories
'''
Fish carriers
Non-fishing vessels nei 146
Unknown 127
Recreational fishing vessels 52
Multipurpose vessels 47
Support Vessels 42
Bunkers 41
Fishery research vessels 37
Multipurpose vessels nei 28
Dredgers 8
None 5
Handliners 4
Liners nei 3
Liners 3
Motherships 2
Seiners 2
Stern trawlers 1
Seiners nei 1
Other line vessels'''
None

In [90]:
# get the important data from the CLAV lists
q = '''
select mmsi, 
group_concat(shiptype) geartype, 
avg(length),avg(tonnage), 
avg(stddev_length), 
avg(stddev_tonnage), 
count(*) number_shiptypes 
from
(select a.mmsi as mmsi, a.shiptype as shiptype, avg(b.length) length, avg(c.tonnage) tonnage, 
stddev(b.length) stddev_length, stddev(c.tonnage) stddev_tonnage
from 
(SELECT mmsi, shipname, 
CASE
  WHEN shiptype = 'Tuna purse seiners' THEN 'Purse seines'
  WHEN shiptype = 'Purse seiners' THEN 'Purse seines'
  WHEN shiptype = 'Tuna longliners' THEN 'Drifting longlines'
  WHEN shiptype = 'Longliners' then 'Drifting longlines'
  WHEN shiptype = 'Trawler' THEN 'Trawlers'
  WHEN shiptype = 'Liners' THEN 'Drifting longlines'
  ELSE shiptype
END shiptype, 
clav_TUVI FROM [world-fishing-827:CLAV_match_results.v7_results]) a
left join
(select length, clav_TUVI from [world-fishing-827:Registry_matching_sources.CLAV_12_14_2015] where length is not null)  b
on a.clav_TUVI = b.clav_TUVI
left join 
(select tonnage, clav_TUVI from [world-fishing-827:Registry_matching_sources.CLAV_12_14_2015] where tonnage is not null)  c
on a.clav_TUVI = c.clav_TUVI
group by mmsi, shiptype)
group by mmsi
'''
clav = Query(q)

Waiting on bqjob_r3b861e6dc13d244b_0000015854b4d4cd_15 ... (0s) Current status: DONE   
Query time: 2.04584884644 seconds.


In [132]:
rows = []
mmsis = []

for c in clav:
    mmsi = c[0]
    geartype = c[1]
    if geartype in clav_rules: # if it is a shiptype we are including
        geartype = clav_rules[geartype] # replace with the right shipname
        length = c[2]
        tonnage = c[3]
        stddev_length = c[4]
        stddev_tonnage = c[5]
        if length !=None: length=float(c[2])
        if tonnage !=None: tonnage = float(c[3])
        if stddev_length !=None: stddev_length = float(c[4])
        if stddev_tonnage !=None: stddev_tonnage = float(c[5])
        if stddev_length !=None and stddev_length/length > .1:
            length = None # if different entries differ by more than 10 percent, ignore
        if stddev_tonnage !=None and stddev_tonnage/tonnage > 1:
            tonnage = None
        num_types = int(c[6])
        rows.append(['Fishing vessel',geartype,geartype,length,tonnage, num_types])
        mmsis.append(mmsi)

In [133]:
clav_frame = pd.DataFrame(rows,index=mmsis,columns=['clav_fishing','clav_label','clav_sublabel', 'clav_length', 'clav_tonnage','num_clav_geartypes'])
clav_frame.index.name = 'mmsi'

In [134]:
clav_frame.to_csv('lists/clav.csv')

In [136]:
# print clav_frame.ix['224025440']
clav_frame.head()

,clav_fishing,clav_label,clav_sublabel,clav_length,clav_tonnage,num_clav_geartypes
mmsi,,,,,,
255500118,Fishing vessel,Pole and line,Pole and line,27.00,121,1
367138310,Fishing vessel,Trollers,Trollers,18.26,32,1
271072420,Fishing vessel,Trollers,Trollers,25.92,111,1
367561420,Fishing vessel,Trollers,Trollers,31.45,177,1
247100040,Fishing vessel,Trawlers,Trawlers,26.20,115,1


In [137]:
clav_frame.groupby('clav_sublabel').count()

,clav_fishing,clav_label,clav_length,clav_tonnage,num_clav_geartypes
clav_sublabel,,,,,
Pole and line,160,160,157,160,160
Set gillnets,23,23,23,17,23
Trawlers,388,388,387,334,388
Trollers,49,49,45,49,49


## Get Vessels that Have Been Matched to the ITU List

In [138]:
# First identify the geartypes

q = '''select itu_Individual_classification, count(*) num from 
[world-fishing-827:Registry_matching_sources.ITU_Dec_2015_Fishing] 
group by itu_Individual_classification
order by 2 desc'''
itu_gears = Query(q)

Waiting on bqjob_r4fb08ee04aaa4736_00000158552a68ca_25 ... (0s) Current status: DONE   
Query time: 2.46540999413 seconds.


In [140]:
for i in itu_gears:
    print i[0],i[1]

Fishing vessel 10352
Trawler 633
None 399
Lobster ship 261
Motor boat 86
Unspecified 57
Passenger ship 47
Cutter 25
Launch 18
Tunny ship 18
Yacht 11
Barge 9
Research ship /Survey ship 9
Reefer 7
Salvage ship 7
Liner 4
Cable ship 3
Coaster 3
Cargo ship 3
Barge carrier 3
Auxiliary ship 3
Sailing ship 2
Whaler 2
RoRo ship 2
Fishing guard 2
Lugger 2
Bulk carrier 2
Factory ship 2
Training ship 2
Air-cushion vehicle 1
Sloop 1
Cargo and passenger 1
Cruiser 1
Supply vessel 1
Lightship 1
Vehicle carrier 1
Dredger 1
Ship used by divers 1
Pontoon 1


In [141]:
# only the following matter

itu_rules_course = {
  'Trawler':'Trawlers',
  'Lobster ship': 'Fixed gear',
  'Fishing vessel':'Fishing vessel'

}

# only the following matter
itu_rules_detail = {
  'Trawler':'Trawlers',
  'Lobster ship': 'Pots and traps',
}

In [142]:
q = ''' SELECT
    mmsi,
    itu_Individual_classification,
    tonnage
  FROM
    [world-fishing-827:Registry_matching_sources.ITU_Dec_2015_Fishing]'''
itu = Query(q)

Waiting on bqjob_r5a385fabeebd21a6_00000158552a9380_26 ... (0s) Current status: DONE   
Query time: 2.39198899269 seconds.


In [143]:
rows = []
mmsis = []

for c in itu:
    if c[1] in itu_rules_course: # if it is a shiptype we are including
        mmsi = c[0]
        geartype_course = itu_rules_course[c[1]] # replace with the right shipname
        try:
            tonnage = float(c[2])
        except:
            tonnage = None  
        if c[1] in itu_rules_detail:
            geartype_detail = itu_rules_detail[c[1]]
        else:
            geartype_detail = None
        rows.append(['Fishing vessel',geartype_course,geartype_detail,tonnage])
        mmsis.append(mmsi)

In [144]:
itu_frame = pd.DataFrame(rows,index=mmsis,columns=['itu_fishing','itu_label','itu_sublabel', 'itu_tonnage'])
itu_frame.index.name = 'mmsi'
itu_frame.to_csv('lists/itu.csv')

In [145]:
itu_frame.head()

,itu_fishing,itu_label,itu_sublabel,itu_tonnage
mmsi,,,,
204211000,Fishing vessel,Fishing vessel,None,NaN
204212000,Fishing vessel,Trawlers,Trawlers,NaN
204213000,Fishing vessel,Fishing vessel,None,NaN
204218000,Fishing vessel,Fishing vessel,None,NaN
204228000,Fishing vessel,Fishing vessel,None,NaN


## Get Vessels that Have Been Matched to the EU List

In [146]:
# first find the geartypes

q = '''select geartype, count(*) from (select a.mmsi, b.Gear_Main_Code geartype, 
b.Loa length, b.Ton_Gt tonnage from 
(SELECT mmsi,row_number FROM [world-fishing-827:EU_match_results.EU_v2]) a
left join [Registry_matching_sources.EU_registry_311215] b
on a.row_number = b.row_number) group by geartype order by 2 desc'''
eu_gear = Query(q)

Waiting on bqjob_r6680736126fd5275_00000158552abef7_27 ... (0s) Current status: DONE   
Query time: 1.29810810089 seconds.


In [147]:
# see http://ec.europa.eu/fisheries/fleet/index.cfm?method=Codification.Cod_gear
eu_gear_codes = {"DRB":"Boat dredges",
"DRH":"Hand dredges operating from a boat",
"HMD":"Mechanised dredges including suction dredges",
"GNC":"Encircling gillnets",
"GND":"Drift nets",
"GNS":"Set gillnets (anchored)",
"GTN":"Combined gillnets-trammel nets",
"GTR":"Trammel nets",
"HAR":"Harpoons",
"LLS":"Set longlines",
"LLD":"Drifting longlines",
"LHM":"Handlines and pole-lines (mechanised)",
"LHP":"Handlines and pole-lines (hand operated)",
"LLD":"Drifting longlines",
"LLS":"Set longlines",
"LTL":"Trolling lines",
"LNB":"Boat-operated lift nets",
"LNS":"Shore-operated stationary lift nets",
"NO":"No gear",
"SB":"Beach seines",
"SDN":"Danish seines",
"SPR":"Pair seines",
"SSC":"Scottish seines",
"LA":"Lampara nets",
"PS":"Purse seines",
"FPO":"Pots and traps",
"OTB":"Bottom otter trawls",
"OTM":"Midwater otter trawls",
"OTT":"Otter twin trawls",
"PTB":"Bottom pair trawls",
"PTM":"Midwater pair trawls",
"TBB":"Beam trawls",
"NK":"Unknown gear"};


for e in eu_gear:
    print e[0],eu_gear_codes[e[0]], e[1]

OTB Bottom otter trawls 3118
PS Purse seines 779
TBB Beam trawls 517
GNS Set gillnets (anchored) 487
OTM Midwater otter trawls 311
LLS Set longlines 241
DRB Boat dredges 229
LLD Drifting longlines 206
FPO Pots and traps 141
GTR Trammel nets 100
OTT Otter twin trawls 66
HMD Mechanised dredges including suction dredges 59
LHP Handlines and pole-lines (hand operated) 48
PTB Bottom pair trawls 43
SDN Danish seines 35
PTM Midwater pair trawls 29
SSC Scottish seines 23
GND Drift nets 16
DRH Hand dredges operating from a boat 14
GTN Combined gillnets-trammel nets 7
LTL Trolling lines 7
SPR Pair seines 3
GNC Encircling gillnets 3
SB Beach seines 2
LNB Boat-operated lift nets 2
LA Lampara nets 1
NO No gear 1


In [148]:
eu_gear2 = {"DRB":"Trawlers",
"DRH":"Trawlers",
"HMD":"Trawlers",
"GNC":"Set gillnets",
"GND":"Fishing vessel",
"GNS":"Set gillnets",
"GTN":"Set gillnets",
"GTR":"Set gillnets",
"HAR":"Fishing vessel",
"LHM":"Pole and line",
"LHP":"Pole and line",
"LLD":"Drifting longlines",
"LLS":"Set longlines",
"LTL":"Trollers",
"LNB":"Fishing vessel",
"LNS":"Fishing vessel",
"NO":"Fishing vessel",
"SB":"Fishing vessel",
"SDN":"Fishing vessel",
"SPR":"Fishing vessel",
"SSC":"Fishing vessel",
"LA":"Set gillnets",
"PS":"Purse seines",
"FPO":"Pots and traps",
"OTB":"Trawlers",
"OTM":"Trawlers",
"OTT":"Trawlers",
"PTB":"Trawlers",
"PTM":"Trawlers",
"TBB":"Trawlers",
"NK":"Unknown gear",
"NO":"No gear"}

In [149]:
# get EU values

q = '''
select mmsi, 
group_concat(geartype) geartype, 
avg(length), 
avg(tonnage),
count(*) num_mmsi,
group_concat(secondary_geartype) secondary_geartype
from
(select a.mmsi mmsi, b.Gear_Main_Code geartype, 
b.Loa length, b.Ton_Gt tonnage, b.Gear_Sec_Code as secondary_geartype
from 
(SELECT mmsi,row_number FROM [world-fishing-827:EU_match_results.EU_v2]) a
left join [Registry_matching_sources.EU_registry_311215] b
on a.row_number = b.row_number)
group by mmsi
having num_mmsi = 1

'''
eu = Query(q)

Waiting on bqjob_r33c3ef672b8324a3_00000158552acd01_28 ... (0s) Current status: DONE   
Query time: 2.46149301529 seconds.


In [151]:
rows = []
mmsis = []

bad_mmsis = []

for c in eu:
    mmsi = c[0]
    if c[1] in eu_gear2 and c[1] != "NO": # if it is a shiptype we are including

        geartype = eu_gear2[c[1]] # replace with the right geartype
        geartype_secondary = eu_gear2[c[5]]
        geartype_primary_course = geartype
        geartype_secondary_course = geartype_secondary
        # make secondary null if it is 
        if geartype[:3] == "Set" or geartype =="Pots and traps":
            geartype_primary_course = "Set gear"   
        if geartype_secondary[:3] == "Set" or geartype_secondary =="Pots and traps":
            geartype_secondary_course = "Set gear"
            
        if geartype_secondary == "No gear" or geartype_secondary == geartype:
            geartype_secondary = None 
            geartype_secondary_course = None
            
    else: # if the primary geartype is not in our list of geartypes, include it as a fishing vessel
        geartype = None
        geartype_secondary = None
        geartype_primary_course = None
        geartype_secondary_course = None
    
    length = c[2]
    tonnage = c[3]
    if length !=None: length=float(c[2])
    if tonnage !=None: tonnage = float(c[3])
    rows.append(["Fishing vessel",geartype_primary_course, geartype, 
                 geartype_secondary_course, geartype_secondary, length, tonnage])
    mmsis.append(mmsi)

In [152]:
headers = ['eu_fishing','eu_label','eu_sublabel','eu_label2','eu_sublabel2',
          'eu_length','eu_tonnage']
eu_frame = pd.DataFrame(rows,index=mmsis,columns=headers)

In [153]:
eu_frame.index.name = 'mmsi'
eu_frame.to_csv('lists/eu.csv')

In [154]:
eu_frame.head()

,eu_fishing,eu_label,eu_sublabel,eu_label2,eu_sublabel2,eu_length,eu_tonnage
mmsi,,,,,,,
227835000,Fishing vessel,Purse seines,Purse seines,None,None,61.00,1383.0
219009244,Fishing vessel,Trawlers,Trawlers,Set gear,Set gillnets,14.01,24.9
247050630,Fishing vessel,Trawlers,Trawlers,None,None,22.00,66.0
261016430,Fishing vessel,Trawlers,Trawlers,None,None,18.92,43.0
237922000,Fishing vessel,Trawlers,Trawlers,Purse seines,Purse seines,21.20,57.0


In [155]:
eu_frame.groupby('eu_sublabel2').count()

,eu_fishing,eu_label,eu_sublabel,eu_label2,eu_length,eu_tonnage
eu_sublabel2,,,,,,
Drifting longlines,71,71,71,71,71,71
Fishing vessel,82,82,82,82,82,82
Pole and line,42,42,42,42,42,42
Pots and traps,106,106,106,106,106,106
Purse seines,319,319,319,319,319,319
Set gillnets,438,438,438,438,438,438
Set longlines,408,408,408,408,408,408
Trawlers,149,149,149,149,149,149
Trollers,14,14,14,14,14,14


## Get Vessels that Have Been Matched to the CCAMLR List

In [156]:
q = '''SELECT geartype, count(*) num 
FROM [world-fishing-827:Registry_matching_sources.CCAMLR_July_2015_with_mmsi] 
group by geartype order by num desc'''
ccamlr = Query(q)

Waiting on bqjob_r4b99d063b6daad0_00000158552b3507_29 ... (0s) Current status: DONE   
Query time: 1.10519504547 seconds.


In [157]:
for c in ccamlr:
    print c[0],c[1]

Longlines (not specified) 23
Midwater trawls (not specified) 9
Other trawls (not specified) 7
Longlines (not specified); Other trawls (not specified) 1
Longlines (not specified); Pots 1


In [158]:
ccamlr_gear = {
'Longlines (not specified)': 'Drifting longlines',
'Midwater trawls (not specified)':"Trawlers", 
'Other trawls (not specified)':'Trawlers' 
}

# ignore the follwoing
# Longlines (not specified); Other trawls (not specified) 1
# Longlines (not specified); Pots 1

In [159]:
q = '''SELECT
    mmsi, geartype, length, tonnage
from  [world-fishing-827:Registry_matching_sources.CCAMLR_July_2015_with_mmsi]
'''
ccamlr = Query(q)

Waiting on bqjob_r12f2070ee4ee67af_00000158552b41f8_30 ... (0s) Current status: DONE   
Query time: 1.09545278549 seconds.


In [161]:
rows = []
mmsis = []

for c in ccamlr:
    if c[1] in ccamlr_gear: # if it is a shiptype we are including
        geartype = ccamlr_gear[c[1]]
    else:
        geartype = "Fishing vessel"
    mmsi = c[0]
    length = c[2]
    tonnage = c[3]
    if length !=None: length=float(c[2])
    if tonnage !=None: tonnage = float(c[3])
    rows.append(['Fishing vessel',geartype,geartype, length,tonnage])
    mmsis.append(mmsi)

In [162]:
ccamlr_frame = pd.DataFrame(rows,index=mmsis,columns=['ccamlr_fishing','ccamlr_label',
                                                      'ccamlr_sublabel','ccamlr_length', 'ccamlr_tonnage'])

In [163]:
ccamlr_frame.index.name = 'mmsi'
ccamlr_frame.to_csv('lists/ccamlr.csv')

## Squid Vessels

In [165]:
# from spreadsheet

squids = [412420249,412420405,412420536,412430920,412439583,412421005,412000812,412200146,
          412206009,412350801,412354049,412354055,412354057,412354058,412354091,412354167,
          412354168,412354445,412354446,412371203,412371204,412371205,412402273,412420008,
          412420109,412420214,412420225,412420228,412420229,412420231,412420242,412420243,
          412420244,412420245,412420246,412420248,412420252,412420253,412420254,412420255,
          412420277,412420278,412420312,412420315,412420317,412420341,412420378,412420382,
          412420383,412420384,412420406,412420409,412420418,412420422,412420423,412420427,
          412420428,412420432,412420439,412420452,412420453,412420454,412420455,412420456,
          412420457,412420458,412420464,412420466,412420467,412420471,412420488,412420493,
          412420494,412420502,412420503,412420504,412420531,412420532,412420533,412420538,
          412420609,412420622,412420623,412420624,412420644,412420646,412420647,412420652,
          412420688,412420691,412420697,412420789,412420798,412420799,412420801,412420816,
          412420822,412420823,412420825,412420872,412420873,412420923,412420925,412420926,
          412420927,412420928,412420933,412420937,412420951,412420955,412420956,412420957,
          412420958,412420959,412420961,412420962,412420963,412420964,412420965,412420966,
          412420968,412420969,412420971,412420972,412420973,412420974,412420975,412420979,
          412420981,412420992,412420994,412421002,412421003,412421004,412421016,412421033,
          412421356,412421357,412422693,412422695,412422696,412422697,412422701,412422706,
          412424491,412424547,412424702,412424703,412430930,412439584,412439586,412439587,
          412439741,412439742,412670670,412679810,412697150,412857888,416562000,416563000,
          440995000,441767000,373039000,356514000,370178000,411164730]
geartypes = [["Fishing vessel","Squid","Squid"] for s in squids]
mmsis = [str(s) for  s in squids]




In [166]:
squid_frame = pd.DataFrame(geartypes,index=mmsis,columns=['squid_fishing',
                                                          'squid_label',
                                                          'squid_sublabel'])
squid_frame.index.name = 'mmsi'
squid_frame.to_csv('lists/squid.csv')

# Reefers

In [167]:
q = '''SELECT
  mmsi,
  "Reefer" geartype,
  AVG(length) length,
  AVG(tonnage) tonnage,
  STDDEV(length) stddev_length,
  STDDEV(tonnage) stddev_tonnage
FROM
  [world-fishing-827:scratch_nate.fish_carrier_list_nate_09_2016_plusD]
GROUP BY
  mmsi,
  geartype
order by mmsi
 '''
reefers = Query(q)

Waiting on bqjob_r50b2d6e52b6806ad_00000158552cd6d4_31 ... (0s) Current status: DONE   
Query time: 1.23183012009 seconds.


In [168]:
rows = []
mmsis = []

for c in reefers:
    mmsi = c[0]
    length = c[2]
    tonnage = c[3]
    geartype = "Reefer"
    stddev_length = c[4]
    stddev_tonnage = c[5]
    if length !=None: length=float(c[2])
    if tonnage !=None: tonnage = float(c[3])
    if stddev_length !=None: stddev_length = float(c[4])
    if stddev_tonnage !=None: stddev_tonnage = float(c[5])
    if stddev_length !=None and stddev_length/length > .1:
        length = None # if different entries differ by more than 10 percent, ignore
    if stddev_tonnage !=None and stddev_tonnage/tonnage > 1:
        tonnage = None
    rows.append(['Nonfishing vessel',geartype,geartype,length,tonnage])
    mmsis.append(mmsi)

In [169]:
reefer_frame = pd.DataFrame(rows,index=mmsis,columns=['reefer_fishing','reefer_label',
                                                      'reefer_sublabel','reefer_length', 'reefer_tonnage'])
reefer_frame.index.name = 'mmsi'
reefer_frame.to_csv('lists/reefer.csv')


## Get Countries from 2015 list
Note that if the vessel wasn't scored for 2015, it has a null value in country in this table

In [126]:
q = '''SELECT
  mmsi,
  iso3
FROM
  [world-fishing-827:scratch_david_mmsi_lists.2015_all_fishing_v4]
WHERE
  label IS NOT NULL
  group by mmsi, iso3'''
countries = Query(q)

Waiting on bqjob_r37a4f6ca5feceede_0000015854bb190b_23 ... (0s) Current status: DONE   
Query time: 3.58140015602 seconds.


In [127]:
mmsis = []
cs = []
for c in countries:
    mmsis.append(c[0])
    cs.append([c[1]])
    
countries_frame = pd.DataFrame(cs,index=mmsis,columns=['country'])
countries_frame.index.name = 'mmsi'
countries_frame.to_csv('lists/countries.csv')


In [356]:
result = eu_frame.join(itu_frame, how='outer')
result = result.join(clav_frame, how='outer')
result = result.join(ccamlr_frame, how='outer')
result = result.join(countries_frame, how='outer')
result = result.join(reefer_frame, how='outer')
result = result.join(squid_frame, how='outer')

# result = pd.concat([result,clav_frame], axis=1, join_axes=[result.index])
# result = result.groupby(result.index).first()
# result = result.drop_duplicates()

In [365]:
result.head()

,eu_geartype,eu_geartype2,eu_length,eu_tonnage,itu_geartype,itu_tonnage,clav_geartype,clav_length,clav_tonnage,ccamlr_geartype,ccamlr_length,ccamlr_tonnage,country,reefer_geartype,reefer_length,reefer_tonnage,squid_geartype
100000005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN
100000018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN
100000046,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN
100000668,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN
100001001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN


In [366]:
result.groupby(['eu_geartype2','clav_geartype']).count()['eu_geartype']

eu_geartype2        clav_geartype     
Drifting longlines  Drifting longlines     71
Fishing vessel      Trawlers                2
Pole and line       Pole and line           9
Purse seines        Pole and line          51
Set gillnets        Drifting longlines      9
                    Set gillnets           12
                    Trawlers                1
Set longlines       Drifting longlines     17
                    Set gillnets            3
Trawlers            Set gillnets            1
                    Trawlers              236
Name: eu_geartype, dtype: int64

In [367]:
rows = []

for i in result.index:
    try:
        mmsi = i
        eu_geartype = result['eu_geartype'].ix[i]
        itu_geartype = result['itu_geartype'].ix[i]
        clav_geartype = result['clav_geartype'].ix[i]
        ccamlr_geartype = result['ccamlr_geartype'].ix[i]
        reefer_geartype = result['reefer_geartype'].ix[i]
        squid_geartype = result['squid_geartype'].ix[i]
        s =  set([eu_geartype, itu_geartype, clav_geartype,ccamlr_geartype,reefer_geartype,squid_geartype])
        if np.nan in s:
            s.remove(np.nan)
        if "Fishing vessel" in s:
            s.remove("Fishing vessel")
        if len(s)!=1:
            geartype = "Fishing vessel"
        else:
            geartype = list(s)[0]
        
        

        # this is a way to figure what what the second gearytpe is
#         if 'ongline' in str(eu_geartype) and geartype != "Fishing vessel":
        if 'Set' in str(eu_geartype) and geartype != "Fishing vessel":
            geartype2 = result['eu_geartype2'].ix[i]
        else: 
            geartype2 = geartype
            
        if geartype[:3]=="Set" or geartype == "Pots and traps":
            geartype = "Set gear"

        eu_tonnage = result['eu_tonnage'].ix[i]
        clav_tonnage = result['clav_tonnage'].ix[i]
        itu_tonnage = result['itu_tonnage'].ix[i]
        ccamlr_tonnage = result['ccamlr_tonnage'].ix[i]
        reefer_tonnage = result['reefer_tonnage'].ix[i]
        tonns = [eu_tonnage,clav_tonnage,itu_tonnage,ccamlr_tonnage,reefer_tonnage]
        tonns = np.array([t for t in tonns if t is not None])
        avg = np.nanmean(tonns)
        stddev = np.nanstd(tonns)
        if stddev/avg > .1:
            tonnage = np.nan
        else:
            tonnage = avg

        if np.isnan(tonnage): tonnage = None

        eu_length= result['eu_length'].ix[i]
        clav_length = result['clav_length'].ix[i]
        ccamlr_length = result['ccamlr_length'].ix[i]
        reefer_length = result['reefer_length'].ix[i]
        length = np.array([eu_length,clav_length,ccamlr_length,reefer_length])
        avg = np.nanmean(length)
        stddev = np.nanstd(length)
        if stddev/avg > .1:
            length = np.nan
        else:
            length = avg
        if np.isnan(length): length = None

        country =  result['country'].ix[i]
        if str(country) == 'nan': country = None

        r = [int(mmsi),geartype,geartype2,length,tonnage,country]    
        rows.append(r)
    except:
        print i 
        break


In [370]:
result.ix['224025440']


eu_geartype                       NaN
eu_geartype2                      NaN
eu_length                         NaN
eu_tonnage                        NaN
itu_geartype                      NaN
itu_tonnage                       NaN
clav_geartype      Drifting longlines
clav_length                        25
clav_tonnage                       85
ccamlr_geartype                   NaN
ccamlr_length                     NaN
ccamlr_tonnage                    NaN
country                           ESP
reefer_geartype                   NaN
reefer_length                     NaN
reefer_tonnage                    NaN
squid_geartype                    NaN
Name: 224025440, dtype: object

In [371]:
header = ['mmsi','geartype','secondary geartype','length','tonnage','country']
with open("fishing_training_20161107.csv", 'wb') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(header)
    writer.writerows(rows)

In [372]:
ft = pd.read_csv('fishing_training_20161107.csv')

In [373]:
ft.groupby(['geartype'])['mmsi'].count()

geartype
Drifting longlines      620
Fishing vessel        40858
Pole and line           120
Purse seines            485
Reefer                  694
Set gear                730
Squid                   164
Trawlers               4286
Trollers                 50
Name: mmsi, dtype: int64

In [374]:
ft.groupby(['secondary geartype'])['mmsi'].count()

secondary geartype
Drifting longlines      620
Fishing vessel        40858
Pole and line           120
Pots and traps          328
Purse seines            485
Reefer                  694
Set gillnets            345
Set longlines            57
Squid                   164
Trawlers               4286
Trollers                 50
Name: mmsi, dtype: int64